In [1]:
import pandas as pd
import numpy as np

In [2]:
# Get data from CSV file
df = pd.read_csv(r"D:\Projects\Streaming_Video_Subscriptions_Analysis\data\Subscription Cohort Analysis Data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3069 entries, 0 to 3068
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            3069 non-null   int64 
 1   created_date           3069 non-null   object
 2   canceled_date          2004 non-null   object
 3   subscription_cost      3069 non-null   int64 
 4   subscription_interval  3069 non-null   object
 5   was_subscription_paid  3069 non-null   object
dtypes: int64(2), object(4)
memory usage: 144.0+ KB


In [3]:
# Display first few rows of CSV file
df.head()

,customer_id,created_date,canceled_date,subscription_cost,subscription_interval,was_subscription_paid
0,154536156,2022-09-01,NaN,39,month,Yes
1,149713408,2022-09-01,2022-09-02,39,month,No
2,153756284,2022-09-01,2022-09-02,39,month,No
3,121253113,2022-09-01,2022-09-23,39,month,Yes
4,154467210,2022-09-01,2023-06-29,39,month,Yes


In [4]:
# Check for null values in the dataset
df.isnull().sum()

customer_id                 0
created_date                0
canceled_date            1065
subscription_cost           0
subscription_interval       0
was_subscription_paid       0
dtype: int64

In [5]:
df["created_date"] = pd.to_datetime(df["created_date"], errors="coerce")
df["canceled_date"] = pd.to_datetime(df["canceled_date"], errors="coerce")
df["subscription_cost"] = df["subscription_cost"].astype(float)

df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3069 entries, 0 to 3068
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   customer_id            3069 non-null   int64         
 1   created_date           3069 non-null   datetime64[ns]
 2   canceled_date          2004 non-null   datetime64[ns]
 3   subscription_cost      3069 non-null   float64       
 4   subscription_interval  3069 non-null   object        
 5   was_subscription_paid  3069 non-null   object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 144.0+ KB


customer_id                 0
created_date                0
canceled_date            1065
subscription_cost           0
subscription_interval       0
was_subscription_paid       0
dtype: int64

In [6]:
# Find duplicate Customer IDs
duplicates = df[df["customer_id"].duplicated(keep=False)]

# Count frequency of each Customer ID
duplicate_count = df["customer_id"].value_counts()
duplicate_count = duplicate_count[duplicate_count > 1]
print(duplicate_count)

customer_id
140062581    3
138725699    3
151633468    3
209743418    3
214682826    3
            ..
160598122    2
116060198    2
203366396    2
138548670    2
182584521    2
Name: count, Length: 185, dtype: int64


In [7]:
# Sort by customer_id and created_date
df = df.sort_values(by=["customer_id", "created_date"])

# For each customer, check if they have multiple subscriptions
df["rejoin_flag"] = df.groupby("customer_id")["created_date"].diff().notna()

# Extract customers who rejoined
rejoined_customers = df[df["rejoin_flag"] == True]["customer_id"].unique()

print(f"Number of customers who rejoined: {len(rejoined_customers)}")
print(f"Sample rejoined customer IDs: {rejoined_customers}")
df

Number of customers who rejoined: 185
Sample rejoined customer IDs: [116060198 119436804 120738319 121253113 121571816 122215146 125038257
 125335731 125828506 127100688 129447970 129747676 130864124 131984682
 132318451 132579012 132631414 132867770 133869989 134652589 134753801
 135092264 135185873 136108848 138535334 138548670 138725699 140062581
 142757196 143111905 144160302 144830936 145096572 146450388 146952862
 147320092 147813476 148568942 148838269 149112756 149303812 149657162
 149856123 150178354 150342446 150521180 150623785 150652956 150779000
 151154171 151513254 151540809 151633468 152223051 152354478 152698255
 152705871 152792045 153186137 153402484 153438676 153876204 154082747
 154535908 154783053 154809306 154901722 154904040 154911229 155088727
 155613371 155869412 156178486 156245288 156403612 156460574 156558244
 156817946 156943008 157374056 157474087 157484010 157663870 157833252
 157883110 157910445 158093021 158575772 158906669 159639128 160598122
 16083365

,customer_id,created_date,canceled_date,subscription_cost,subscription_interval,was_subscription_paid,rejoin_flag
1796,111394466,2023-04-20,2023-06-22,39.0,month,Yes,False
2481,112301350,2023-07-06,NaT,39.0,month,Yes,False
1789,112792926,2023-04-20,NaT,39.0,month,Yes,False
451,113181220,2022-11-05,2023-07-23,39.0,month,Yes,False
544,114435366,2022-11-18,2022-11-18,39.0,month,Yes,False
...,...,...,...,...,...,...,...
3055,221100604,2023-09-06,NaT,39.0,month,Yes,False
3052,221123938,2023-09-06,NaT,39.0,month,Yes,False
3065,221130434,2023-09-07,NaT,39.0,month,Yes,False
3064,221145374,2023-09-07,NaT,39.0,month,Yes,False


In [8]:
# Filter rows where canceled_date is not null
filtered = df[df["canceled_date"].notna()]

# Check condition
result = (filtered["canceled_date"] >= filtered["created_date"]).all()

print("All non-null canceled dates are greater than subscription dates:", result)


All non-null canceled dates are greater than subscription dates: True


In [9]:
df["status"] = np.where(df["canceled_date"].isna(), "Active", "Canceled")
df

,customer_id,created_date,canceled_date,subscription_cost,subscription_interval,was_subscription_paid,rejoin_flag,status
1796,111394466,2023-04-20,2023-06-22,39.0,month,Yes,False,Canceled
2481,112301350,2023-07-06,NaT,39.0,month,Yes,False,Active
1789,112792926,2023-04-20,NaT,39.0,month,Yes,False,Active
451,113181220,2022-11-05,2023-07-23,39.0,month,Yes,False,Canceled
544,114435366,2022-11-18,2022-11-18,39.0,month,Yes,False,Canceled
...,...,...,...,...,...,...,...,...
3055,221100604,2023-09-06,NaT,39.0,month,Yes,False,Active
3052,221123938,2023-09-06,NaT,39.0,month,Yes,False,Active
3065,221130434,2023-09-07,NaT,39.0,month,Yes,False,Active
3064,221145374,2023-09-07,NaT,39.0,month,Yes,False,Active


In [10]:
total_customers = df["customer_id"].nunique()
total_customers

2877

In [11]:
active_customers = df.loc[df["status"] == "Active", "customer_id"].nunique()
canceled_customers = df.loc[df["status"] == "Canceled", "customer_id"].nunique()
churn_rate = canceled_customers / total_customers
churn_rate

0.6736183524504692

In [12]:
rejoined_customers = df.loc[df["rejoin_flag"] == True, "customer_id"].nunique()
rejoin_rate = rejoined_customers / total_customers
rejoin_rate

0.0643030935001738

In [13]:
monthly_recurring_revenue = active_customers * np.mean(df["subscription_cost"])
monthly_recurring_revenue

41535.0